# Natural Statistics Cross-linguistic: 

#### Proportion of single-word utterances analysis

----

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "data_proc")
import contingent_extraction
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load and clean data
rand_dat_inc = pd.read_csv("../data/rand_dat_inc_master.csv",index_col=0,low_memory=False)
rand_dat_inc=rand_dat_inc[rand_dat_inc["language"]!="ara"]
rand_dat_inc=rand_dat_inc[(rand_dat_inc["target_child_age"]>=5) & (rand_dat_inc["target_child_age"]<=30)]
rand_dat_inc_cg = rand_dat_inc[rand_dat_inc["caregiver"]=="caregiver"]

rand_dat_inc_cg["contingent"] = np.where(rand_dat_inc_cg["contingent"]==1, "contingent", "non-contingent")

rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"].notna()]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="xxx"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="yyy"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="www"]

rand_dat_inc_cg["swu"]=np.where(rand_dat_inc_cg["num_tokens"]==1,1,0)

In [3]:
# add play context and year of study

play_context = pd.read_csv("../data/context_data.csv")
play_context = play_context.rename(columns={"Corpus": "corpus_name"})

# print(play_context.to_markdown())

rand_dat_inc_cg = rand_dat_inc_cg.merge(play_context,on='corpus_name')

rand_dat_inc_cg["context"] = rand_dat_inc_cg["Location"] + rand_dat_inc_cg["Activity"]

rand_dat_inc_cg["context"] = rand_dat_inc_cg["context"].replace({"HomeBook-reading":"Home: book reading",
                                                                 "HomeInterview/Unstructured":"Home: interview/unstructured",
                                                                 "HomeNaN":"Home: unreported",
                                                                 "HomeOther":"Home: other",
                                                                 "HomeUnstructured":"Home: unstructured",
                                                                 "LabOther":"Lab: other",
                                                                 "LabTabletop play":"Lab: tabletop play",
                                                                 "LabInterview/Unstructured":"Lab: interview/unstructured",
                                                                 "LabUnstructured":"Lab: unstructured",
                                                                 np.nan:"Unreported",
                                                                 "OtherUnstructured":"Other: unstructured"})

# year of study
corpora_year = pd.read_csv("../data/corpora_year.csv")
corpora_year = corpora_year.rename(columns={"Corpora": "corpus_name"})
corpora_year = corpora_year[["corpus_name", "Year collected"]]

rand_dat_inc_cg = rand_dat_inc_cg.merge(corpora_year,on='corpus_name')

In [4]:
%load_ext rpy2.ipython

In [5]:
%%R -i rand_dat_inc_cg

library("lme4")
library("repr")
library("knitr")
library("broom")
library("emmeans")
library("tidyverse")
library("kableExtra")

options(repr.plot.width=6, repr.plot.height=12, scipen=999)

R[write to console]: Loading required package: Matrix

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()

R[write to console]: 
Attaching package: ‘kableExtra’


R[write to console]: The following object is masked from ‘package:dplyr’:

    group_rows




In [6]:
%%R -o rand_dat_inc_cg

# ---- create caregiver type categories

caregiver_type <- rand_dat_inc_cg %>%
  group_by(transcript_id) %>%
  summarise(
    caregiver_type = case_when(
      all(speaker_role == "Mother") ~ "Mother only",
      all(speaker_role == "Father") ~ "Father only",
      any(speaker_role %in% c("Mother", "Father")) ~ "Mother & Father",
      TRUE ~ "Unknown"
    )
  )

# inspect data:
# caregiver_type %>%
#     kbl(format="pipe")
    
# ggplot(caregiver_type, aes(x = 1, y = caregiver_type, fill = factor(caregiver_type))) + 
#   geom_col() +
#   coord_polar(theta = "y") +
#   theme_void()

rand_dat_inc_cg <- rand_dat_inc_cg %>%
  left_join(caregiver_type)

Joining with `by = join_by(transcript_id)`


In [7]:
rand_swu_stats = (rand_dat_inc_cg.groupby(["Language_name","target_child_id","transcript_id","contingent"])
                                  .swu
                                  .agg(["mean"])
                                  .reset_index())
rand_swu_sumstats =  rand_swu_stats.rename({'mean': 'means'}, axis=1)

In [8]:
%%R -i rand_swu_sumstats

# ^import rand_swu_sumstats into R

NULL


In [9]:
%%R -o rand_swu_sumstats

rand_swu_sumstats <- rand_swu_sumstats %>%
    left_join(caregiver_type)

Joining with `by = join_by(transcript_id)`


In [10]:
# save data to file:
rand_swu_sumstats.to_csv("../data/rand_swu_sumstats.csv")

Simple plot

In [11]:
%%R

xlabs <- c("C", "NC")

# # ara_label <- data.frame(means=c(.9),contingent = c(1.5),language="ara")
deu_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="German")
# deu_ns_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="English")
est_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Estonian")
# est_ns_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Korean")
nor_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Norwegian")
pol_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Portuguese")
spa_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Swedish")
zho_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_swu_sumstats, aes(x = contingent, y = means, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=8,color="black") +
     geom_text(data = eng_label,label = "***",size=8,color="black") +  
     geom_text(data = est_label,label = "ns", size=4,color="black",fontface = "italic") +  
     geom_text(data = fas_label,label = "ns", size=4,color="black",fontface = "italic") + 
     geom_text(data = fra_label,label = "***",size=8,color="black") +  
     geom_text(data = hrv_label,label = "***",size=8,color="black") + 
     geom_text(data = jpn_label,label = "***",size=8,color="black") + 
     geom_text(data = kor_label,label = "***",size=8,color="black") +  
     geom_text(data = nor_label,label = "**",size=8,color="black") +  
     geom_text(data = pol_label,label = "ns", size=4,color="black",fontface = "italic") +    
     geom_text(data = por_label,label = "***",size=8,color="black") +  
     geom_text(data = spa_label,label = "***",size=8,color="black") + 
     geom_text(data = swe_label,label = "***",size=8,color="black") + 
     geom_text(data = zho_label,label = "ns", size=4, color="black",fontface = "italic") +
     ylim(0, .5) +
     labs(tag="C",
          y = "Proportion of Single Word Utterances",
          x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=16),
           axis.text.x = element_text(vjust = 0.5, hjust = 0.5),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
      ggsave("../figures/token_rand_swu.pdf", width = 11.7, height = 6.2)


for manuscript

In [12]:
%%R -i rand_swu_sumstats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# # ara_label <- data.frame(means=c(.9),contingent = c(1.5),language="ara")
deu_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="German")
# deu_ns_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="English")
est_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Estonian")
# est_ns_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Korean")
nor_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Norwegian")
pol_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Portuguese")
spa_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(means=c(.47),contingent = c(1.5),Language_name="Swedish")
zho_label <- data.frame(means=c(.5),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_swu_sumstats, aes(x = contingent, y = means, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=8,color="black") +
     geom_text(data = eng_label,label = "***",size=8,color="black") +  
     geom_text(data = est_label,label = "ns", size=4,color="black",fontface = "italic") +  
     geom_text(data = fas_label,label = "ns", size=4,color="black",fontface = "italic") + 
     geom_text(data = fra_label,label = "***",size=8,color="black") +  
     geom_text(data = hrv_label,label = "***",size=8,color="black") + 
     geom_text(data = jpn_label,label = "***",size=8,color="black") + 
     geom_text(data = kor_label,label = "***",size=8,color="black") +  
     geom_text(data = nor_label,label = "**",size=8,color="black") +  
     geom_text(data = pol_label,label = "ns", size=4,color="black",fontface = "italic") +    
     geom_text(data = por_label,label = "***",size=8,color="black") +  
     geom_text(data = spa_label,label = "***",size=8,color="black") + 
     geom_text(data = swe_label,label = "***",size=8,color="black") + 
     geom_text(data = zho_label,label = "ns", size=4, color="black",fontface = "italic") +
     ylim(0, .5) +
     labs(tag="C",
          y = "Proportion of Single Word Utterances",
          x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=11.5),
           axis.text.x = element_text(vjust = 0.5, hjust=0.5),
           legend.position="none")

      ggsave("../figures/figure_2_C.pdf", width = 11.5, height = 4.2)

----

#### SWU mixed models

Extract effect sizes to file

In [ ]:
deu=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="deu"]
eng=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="eng"]
est=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="est"]
fas=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="fas"]
fra=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="fra"]
hrv=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="hrv"]
jpn=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="jpn"]
kor=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="kor"]
nor=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="nor"]
pol=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="pol"]
por=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="por"]
spa=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="spa"]
swe=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="swe"]
zho=rand_dat_inc_cg[['language','swu','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="zho"]

In [ ]:
%%R

library("lme4")
library("broom")
library("emmeans")
library("lmerTest")
library("tidyverse")

options(scipen = 999)

effect_sizes <- data.frame(matrix(ncol = 2, nrow = 0))
cols <- c("Language_name", "rand_effect_size")
colnames(effect_sizes) <- cols

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step


R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
✔ purrr   0.3.5      

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()



In [ ]:
%%R -i deu

lm2_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=deu)
emm2_1<-emmeans(lm2_1,pairwise~contingent)
pval<-summary(emm2_1$contrasts)$p.value
print(c(emm2_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# plot(emm2_1)
# summary(emmeans(lm2_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm2_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

deu_lname <- deu$language[1]

deu_eff <- eff_size(emm2_1,sigma = sigma(lm2_1), edf = df.residual(lm2_1))

deu_eff <- summary(deu_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(deu_lname,deu_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 4452' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 4452)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 4452' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 4452)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate     SE  df z.ratio p.value
 contingent - (non-contingent)    0.081 0.0125 Inf   6.498  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.00000000008122039

[1] 0.000000001137085


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804


In [ ]:
%%R -i eng

lm3_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=eng, REML= FALSE)
emm3_1<-emmeans(lm3_1,pairwise~contingent)
pval<-summary(emm3_1$contrasts)$p.value
print(c(emm3_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm3_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm3_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

eng_lname <- eng$language[1]

eng_eff <- eff_size(emm3_1,sigma = sigma(lm3_1), edf = df.residual(lm3_1))

eng_eff <- summary(eng_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(eng_lname,eng_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 113356' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 113356)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 113356' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 113356)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate      SE  df z.ratio p.value
 contingent - (non-contingent)    0.107 0.00351 Inf  30.329  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004692424

[1] 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006569394


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672


In [ ]:
%%R -i est

lm4_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=est, REML= FALSE)
emm4_1<-emmeans(lm4_1,pairwise~contingent)
pval<-summary(emm4_1$contrasts)$p.value
print(c(emm4_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm4_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm4_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

est_lname <- est$language[1]

est_eff <- eff_size(emm4_1,sigma = sigma(lm4_1), edf = df.residual(lm4_1))

est_eff <- summary(est_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(est_lname,est_eff)
effect_sizes

[[1]]
 contrast                      estimate     SE   df t.ratio p.value
 contingent - (non-contingent)   0.0427 0.0148 2309   2.875  0.0041

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.004079875

[1] 0.05711826


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875


In [ ]:
%%R -i fas

lm5_1 <- lmer(swu ~ contingent + (1|transcript_id),data=fas, REML= FALSE)
emm5_1<-emmeans(lm5_1,pairwise~contingent)
pval<-summary(emm5_1$contrasts)$p.value
print(c(emm5_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm5_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm5_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

fas_lname <- fas$language[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(fas_lname,NaN)
effect_sizes

[[1]]
 contrast                      estimate     SE  df t.ratio p.value
 contingent - (non-contingent)    0.125 0.0463 641   2.702  0.0071

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.007072117

[1] 0.09900964
  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875
4           fas               NaN


In [ ]:
%%R -i fra
lm6_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=fra, REML= FALSE)
emm6_1<-emmeans(lm6_1,pairwise~contingent)
pval<-summary(emm6_1$contrasts)$p.value
print(c(emm6_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm6_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm6_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

fra_lname <- fra$language[1]

fra_eff <- eff_size(emm6_1,sigma = sigma(lm6_1), edf = df.residual(lm6_1))

fra_eff <- summary(fra_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(fra_lname,fra_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 21241' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 21241)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 21241' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 21241)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate      SE  df z.ratio p.value
 contingent - (non-contingent)   0.0509 0.00654 Inf   7.779  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.000000000000007299281

[1] 0.0000000000001021899


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875
4           fas               NaN
5           fra 0.129286127432911


In [ ]:
%%R -i hrv

lm7_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=hrv, REML= FALSE)
emm7_1<-emmeans(lm7_1,pairwise~contingent)
pval<-summary(emm7_1$contrasts)$p.value
print(c(emm7_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm7_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm7_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

hrv_lname <- hrv$language[1]

hrv_eff <- eff_size(emm7_1,sigma = sigma(lm7_1), edf = df.residual(lm7_1))

hrv_eff <- summary(hrv_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(hrv_lname,hrv_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 6251' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 6251)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 6251' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 6251)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate     SE  df z.ratio p.value
 contingent - (non-contingent)   0.0693 0.0107 Inf   6.463  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.0000000001023428

[1] 0.000000001432799


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875
4           fas               NaN
5           fra 0.129286127432911
6           hrv 0.169079925211206


In [ ]:
%%R -i jpn

lm8_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=jpn, REML= FALSE)
emm8_1<-emmeans(lm8_1,pairwise~contingent)
pval<-summary(emm8_1$contrasts)$p.value
print(c(emm8_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm8_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm8_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

jpn_lname <- jpn$language[1]

jpn_eff <- eff_size(emm8_1,sigma = sigma(lm8_1), edf = df.residual(lm8_1))

jpn_eff <- summary(jpn_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(jpn_lname,jpn_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 25124' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 25124)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 25124' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 25124)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate      SE  df z.ratio p.value
 contingent - (non-contingent)    0.175 0.00634 Inf  27.631  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004779791

[1] 0.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006691708


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875
4           fas               NaN
5           fra 0.129286127432911
6           hrv 0.169079925211206
7           jpn 0.370538605687043


In [ ]:
%%R -i kor

lm9_1 <- lmer(swu ~ contingent + (1|transcript_id),data=kor, REML= FALSE)
emm9_1<-emmeans(lm9_1,pairwise~contingent)
pval<-summary(emm9_1$contrasts)$p.value
print(c(emm9_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm9_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm9_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

kor_lname <- kor$language[1]

kor_eff <- eff_size(emm9_1,sigma = sigma(lm9_1), edf = df.residual(lm9_1))

kor_eff <- summary(kor_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(kor_lname,kor_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 4615' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 4615)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 4615' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 4615)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate     SE  df z.ratio p.value
 contingent - (non-contingent)   0.0848 0.0106 Inf   8.024  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.000000000000001023043

[1] 0.00000000000001432261


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875
4           fas               NaN
5           fra 0.129286127432911
6           hrv 0.169079925211206
7           jpn 0.370538605687043
8           kor  0.25819504353917


In [ ]:
%%R -i nor

lm10_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=nor, REML= FALSE)
emm10_1<-emmeans(lm10_1,pairwise~contingent)
pval<-summary(emm10_1$contrasts)$p.value
print(c(emm10_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm10_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm10_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

nor_lname <- nor$language[1]

nor_eff <- eff_size(emm10_1,sigma = sigma(lm10_1), edf = df.residual(lm10_1))

nor_eff <- summary(nor_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(nor_lname,nor_eff)
effect_sizes

R[write to console]: boundary (singular) fit: see help('isSingular')



[[1]]
 contrast                      estimate     SE   df t.ratio p.value
 contingent - (non-contingent)    0.101 0.0211 1923   4.784  <.0001

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.000001850965

[1] 0.00002591352


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1           deu 0.203791769489804
2           eng 0.267969063164672
3           est 0.125671598106875
4           fas               NaN
5           fra 0.129286127432911
6           hrv 0.169079925211206
7           jpn 0.370538605687043
8           kor  0.25819504353917
9           nor 0.250109617249279


In [ ]:
%%R -i pol
 
# simple linear model (no random effects, because only 1 transcript from 1 sub)

lm11_1 <- lm(swu ~ contingent,data=pol, REML= FALSE)
emm11_1<-emmeans(lm11_1,pairwise~contingent)
pval<-summary(emm11_1$contrasts)$p.value
print(c(emm11_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm11_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm11_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

pol_lname <- pol$language[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(pol_lname,NaN)
effect_sizes

[[1]]
 contrast                      estimate    SE df t.ratio p.value
 contingent - (non-contingent)    0.045 0.112 50   0.401  0.6898


[[2]]
[1] 0.689809

[1] 1
   Language_name  rand_effect_size
1            deu 0.203791769489804
2            eng 0.267969063164672
3            est 0.125671598106875
4            fas               NaN
5            fra 0.129286127432911
6            hrv 0.169079925211206
7            jpn 0.370538605687043
8            kor  0.25819504353917
9            nor 0.250109617249279
10           pol               NaN


In [ ]:
%%R -i por

lm12_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=por, REML= FALSE)
emm12_1<-emmeans(lm12_1,pairwise~contingent)
pval<-summary(emm12_1$contrasts)$p.value
print(c(emm12_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm12_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm12_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

por_lname <- por$language[1]

por_eff <- eff_size(emm12_1,sigma = sigma(lm12_1), edf = df.residual(lm12_1))

por_eff <- summary(por_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(por_lname,por_eff)
effect_sizes

[[1]]
 contrast                      estimate     SE   df t.ratio p.value
 contingent - (non-contingent)   0.0698 0.0142 2661   4.905  <.0001

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.00000099005

[1] 0.0000138607


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name  rand_effect_size
1            deu 0.203791769489804
2            eng 0.267969063164672
3            est 0.125671598106875
4            fas               NaN
5            fra 0.129286127432911
6            hrv 0.169079925211206
7            jpn 0.370538605687043
8            kor  0.25819504353917
9            nor 0.250109617249279
10           pol               NaN
11           por  0.19962474315635


In [ ]:
%%R -i spa

lm13_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=spa, REML= FALSE)
emm13_1<-emmeans(lm13_1,pairwise~contingent)
pval<-summary(emm13_1$contrasts)$p.value
print(c(emm13_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm13_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm13_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

spa_lname <- spa$language[1]

spa_eff <- eff_size(emm13_1,sigma = sigma(lm13_1), edf = df.residual(lm13_1))

spa_eff <- summary(spa_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(spa_lname,spa_eff)
effect_sizes

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 4375' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 4375)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 4375' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 4375)' or larger];
but be warned that this may result in large computation time and memory use.



[[1]]
 contrast                      estimate     SE  df z.ratio p.value
 contingent - (non-contingent)   0.0907 0.0124 Inf   7.301  <.0001

Degrees-of-freedom method: asymptotic 

[[2]]
[1] 0.000000000000286318

[1] 0.000000000004008452


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name  rand_effect_size
1            deu 0.203791769489804
2            eng 0.267969063164672
3            est 0.125671598106875
4            fas               NaN
5            fra 0.129286127432911
6            hrv 0.169079925211206
7            jpn 0.370538605687043
8            kor  0.25819504353917
9            nor 0.250109617249279
10           pol               NaN
11           por  0.19962474315635
12           spa 0.223848639903587


In [ ]:
%%R -i swe


lm14_1 <- lmer(swu ~ contingent + (1|target_child_id) + (1|transcript_id),data=swe, REML= FALSE)
emm14_1<-emmeans(lm14_1,pairwise~contingent)
pval<-summary(emm14_1$contrasts)$p.value
print(c(emm14_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm14_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm14_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

swe_lname <- swe$language[1]

swe_eff <- eff_size(emm14_1,sigma = sigma(lm14_1), edf = df.residual(lm14_1))

swe_eff <- summary(swe_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(swe_lname,swe_eff)
effect_sizes

[[1]]
 contrast                      estimate     SE   df t.ratio p.value
 contingent - (non-contingent)    0.136 0.0172 2715   7.892  <.0001

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.000000000000004279189

[1] 0.00000000000005990865


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name  rand_effect_size
1            deu 0.203791769489804
2            eng 0.267969063164672
3            est 0.125671598106875
4            fas               NaN
5            fra 0.129286127432911
6            hrv 0.169079925211206
7            jpn 0.370538605687043
8            kor  0.25819504353917
9            nor 0.250109617249279
10           pol               NaN
11           por  0.19962474315635
12           spa 0.223848639903587
13           swe 0.316404337565653


In [ ]:
%%R -i zho

lm15_1 <- lmer(swu ~ contingent + (1|transcript_id),data=zho, REML= FALSE)
emm15_1<-emmeans(lm15_1,pairwise~contingent)
pval<-summary(emm15_1$contrasts)$p.value
print(c(emm15_1$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm15_1,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm15_1,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

zho_lname <- zho$language[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(zho_lname,NaN)
effect_sizes

[[1]]
 contrast                      estimate     SE  df t.ratio p.value
 contingent - (non-contingent)    0.087 0.0412 379   2.113  0.0353

Degrees-of-freedom method: kenward-roger 

[[2]]
[1] 0.03527606

[1] 0.4938648
   Language_name  rand_effect_size
1            deu 0.203791769489804
2            eng 0.267969063164672
3            est 0.125671598106875
4            fas               NaN
5            fra 0.129286127432911
6            hrv 0.169079925211206
7            jpn 0.370538605687043
8            kor  0.25819504353917
9            nor 0.250109617249279
10           pol               NaN
11           por  0.19962474315635
12           spa 0.223848639903587
13           swe 0.316404337565653
14           zho               NaN


In [ ]:
%%R
write.csv(x=effect_sizes,'../data/SWU_effect_sizes.csv', row.names = FALSE)